In [30]:
import math

# Центр голосования

Создаёт две пары ключей — приватную $(d, n)$ хранит только у себя, а публичную $(e, n)$ раздаёт голосующим

In [31]:
(d, n) = (41,   9991) # (d, n) -- Приватный
(e, n) = (1433, 9991) # (e, n) -- Публичный

# Голоса будем хранить в списке g:
golosa = []

# Голосующщие

Избиратель выбирает свой вариант: «ЗА», «ПРОТИВ», «ВОЗДЕРЖАЛСЯ».

Голос шифруется с использованием публичного ключа центра голосования, чтобы никто кроме центра не мог его расшифровать.

Часто используется затенение (blinding) — добавляется случайный множитель, чтобы центр голосования не видел конкретный голос, но мог его потом подсчитать:

$$g_i = (a_i * k_i)^e \mod n,$$

где:
- $a_i$ — выбранный вариант (2 за, 3 против, 1 воздержался (не имеет веса));
- $k_i$ — случайный коэффициент (затеняющий коэффициент);
- $(e,n)$ — публичный ключ.

In [32]:
# Выбираем затеняющий коэффициент k 
k = input('Введите затеняющий коэффициент: ')
k = int(k) if k else 23

# k должно НЕ ДЕЛИТЬСЯ НИ НА 2, НИ НА 3!
if (k % 2 == 0) or (k % 3 == 0):
    print(f'Ошибка -- k = {k} не должно делиться ни на 2, ни на 3!')


while True:
    # a = 2 -- ЗА либо 3 -- ПРОТИВ
    a = input('Введите 2 (ЗА), 3 (ПРОТИВ), 1 (ВОЗДЕРЖ.) либо 0 (ВЫХОД): ')
    a = int(a) if a else 0

    if a == 0:
        break

    # Вычисляется результат затенения
    b = a * k

    # b должно быть СТРОГО МЕНЬШЕ, чем n!
    if (b >= n):
        print(f'Ошибка -- b = {b} должно быть < n = {n}')

    golosa.append(b ** e % n)

    # g_i отправляется в реестр, где i -- номер проголосовавшего
    print(f'g = {golosa}')

g = [1646]
g = [1646, 1646]
g = [1646, 1646, 5476]
g = [1646, 1646, 5476, 1646]
g = [1646, 1646, 5476, 1646, 7667]


# Подсчёт голосов

Центр использует приватный ключ $(d,n)$, чтобы расшифровать голоса:

$$b_i = g_i * d \mod n$$

Далее, из произведения всех голосов, можно подсчитать коэффициент всех голосов:

$$F = \prod_i * \ b_i$$


Наконец, с помощью разложения $F$ на множители $2$ и $3$, извлекается количество голосов «ЗА» и «ПРОТИВ» соответственно.

In [38]:
# G = math.prod(golosa)

# F = (G ** d) % n # = math.prod(g) ** d % n = math.prod(g ** d) % n
#                  # = math.prod(b) = math.prod(a * k)
#                  # = (2 ** r) * (3 ** p) * K, где K = math.prod(k)

# golosa содержит зашифрованные голоса, расшифруем их
razgolos = [b ** d % n for b in golosa]
print(razgolos)

# Посчитаем произведение расшифрованных голосов
F = math.prod(razgolos)
print(f'Коэффициент всех голосов F = {F}')

# Теперь выполняем разложение F на 2 (ЗА) и на 3 (ПРОТИВ)
# и считаем, сколько раз коэффициент разложился:
za = 0; protiv = 0

c = F

while (c % 2 == 0):
    c //= 2
    za += 1

while (c % 3 == 0):
    c //= 3
    protiv += 1

print(f'\n=== ИТОГИ ГОЛОСОВАНИЯ ===')
print(f'ЗА: {za} \nПРОТИВ: {protiv} \nВСЕГО: {len(golosa)}')

# r_i * p_i * K_i * g_i -- Для проверки

[86, 86, 129, 86, 43]
Коэффициент всех голосов F = 3528202632

=== ИТОГИ ГОЛОСОВАНИЯ ===
ЗА: 3 
ПРОТИВ: 1 
ВСЕГО: 5


# Проверка результатов

1. Считается $F$ по голосам в центре
2. Считается $G$ по голосам из реестра
3. Проверяется, что $G = (F^e) % n$

In [40]:
# Произведение всех расшифрованных голосов
F_check = math.prod(razgolos) % n

# Произведение всех зашифрованных голосов
G_check = math.prod(golosa) % n

# Проверка
if G_check == F_check ** e % n:
    print('Проверка пройдена! Результаты центра соответствуют реестру')
else:
    print('Фальсификация!!! Результаты центра не соответствуют реестру!')

Проверка пройдена! Результаты центра соответствуют реестру
